In [ ]:
!pip uninstall -y numpy scipy albumentations scikit-image
!pip install numpy==1.26.4 scipy==1.11.4 albumentations==1.3.1 scikit-image==0.21.0


In [1]:
!pip install ultralytics --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.9 MB/s eta 0:00:00a 0:00:01


In [3]:
import os
import random
import shutil
import stat
import numpy as np
import torch
import cv2
import albumentations as A
from ultralytics import YOLO

# =========================================================
# ✅ Reproducibility: Set random seeds (consistent everywhere)
# =========================================================
def set_seed(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # Ensure deterministic operations on GPU (may slow training)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED = 42
set_seed(SEED)
print(f"✅ Random seed set to {SEED}")

# =========================================================
# Versions
# =========================================================
print("Albumentations:", A.__version__)
print("Torch:", torch.__version__)

# =========================================================
# Paths & dataset writability handling (Kaggle-safe)
# =========================================================
orig_data_yaml = "/kaggle/input/violenceweapondetection/data.yaml"
# Destination (writable) dataset root for faster caching / saving
writable_dataset_root = "/kaggle/working/violenceweapondetection"

def ensure_writable_dataset(src_yaml_path, dst_root):
    """
    If the dataset path is not writable (e.g. /kaggle/input), copy dataset to a writable location.
    Returns path to data.yaml to use (and creates dst_root if needed).
    """
    src_root = os.path.dirname(src_yaml_path)
    # If src is already writable, return original
    try:
        testfile = os.path.join(src_root, ".writetest")
        with open(testfile, "w") as f:
            f.write("test")
        os.remove(testfile)
        print("Dataset directory is writable — no copy needed.")
        return src_yaml_path
    except Exception:
        print("Dataset directory not writable — copying to a writable location (this may take time).")
    # Copy (skip if already copied)
    if os.path.exists(dst_root):
        print("Writable copy already exists.")
    else:
        shutil.copytree(src_root, dst_root)
        # make sure permissions allow write
        for root, dirs, files in os.walk(dst_root):
            for d in dirs:
                os.chmod(os.path.join(root, d), stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP)
            for f in files:
                os.chmod(os.path.join(root, f), stat.S_IRUSR | stat.S_IWUSR)
        print(f"Copied dataset to {dst_root}")
    # update yaml path (same filename inside new root)
    return os.path.join(dst_root, os.path.basename(src_yaml_path))

data_yaml = ensure_writable_dataset(orig_data_yaml, writable_dataset_root)

# =========================================================
# ✅ Custom CCTV Augmentation Pipeline (Albumentations)
# NOTE: remove ToTensorV2 — let Ultralytics dataset handle conversion
# =========================================================
train_transform = A.Compose([
    # Geometric transformations
    A.RandomResizedCrop(height=640, width=640, scale=(0.7, 1.0), ratio=(0.75, 1.33), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15,
                       border_mode=cv2.BORDER_CONSTANT, p=0.8),

    # Brightness / contrast / color (CCTV-like)
    A.RandomBrightnessContrast(p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=15, p=0.6),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.4),

    # Blur / Noise (CCTV noise)
    A.MotionBlur(p=0.3),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.GaussNoise(var_limit=(5, 25), p=0.3),

    # Occlusions
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.4),
], p=1.0)

# =========================================================
# ✅ Load YOLOv11s Pretrained Model
# =========================================================
model = YOLO("yolo11s.pt")  # will auto-download if not exists

# Force Ultralytics to not use internal 'randaugment' that may call ImageCompression with unsupported args
# (this avoids the ImageCompression.__init__() got unexpected keyword 'quality_range' warning)
model.overrides["auto_augment"] = None

# YOLO augmentation controls (kept)
model.overrides["augment"] = True
model.overrides["rect"] = False
model.overrides["mosaic"] = 0.3
model.overrides["mixup"] = 0.2
model.overrides["copy_paste"] = 0.0

# =========================================================
# HOOK Albumentations into YOLO dataset
# Use on_fit_start so dataset exists and we can set transforms before epochs run
# =========================================================
def custom_dataloader_hook(trainer):
    """
    trainer.train_loader.dataset is expected to be the Ultralytics dataset instance.
    Setting dataset.transforms to an Albumentations Compose that accepts/returns numpy arrays
    (Ultralytics will handle conversion to tensors internally).
    """
    try:
        ds = trainer.train_loader.dataset
        ds.transforms = train_transform
        print("✅ Injected custom Albumentations transforms into dataset.")
    except Exception as e:
        # In case API differs, print the error so you can adjust.
        print("⚠️ Could not inject transforms into dataset (exception):", e)

# Register callback
model.add_callback("on_fit_start", custom_dataloader_hook)

# =========================================================
# TRAIN
# - Pass seed explicitly
# - disable internal auto_augment via overrides above
# =========================================================
save_dir = "/kaggle/working/yolov11s_cctv_aug_runs"

results = model.train(
    data=data_yaml,
    epochs=300,
    imgsz=640,
    batch=16,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=save_dir,
    name="yolov11s_cctv_aug",
    workers=2,

    # Repro / hyperparams
    seed=SEED,
    deterministic=True,

    # Best CCTV hyperparameters
    lr0=0.0025,
    lrf=0.01,
    optimizer="AdamW",
    momentum=0.90,
    weight_decay=0.0004,
    warmup_epochs=3.0,
    patience=25,

    mosaic=0.3,
    mixup=0.2,
    copy_paste=0.0,
    augment=True,

    val=True,
    pretrained=True,
    verbose=True,
    exist_ok=True,

    save=True
)

# =========================================================
# EVALUATION
# =========================================================
metrics = model.val()
print("Validation Metrics:")
try:
    print(f"mAP50-95: {metrics.box.map:.4f}")
    print(f"Precision: {metrics.box.pr:.4f}")
    print(f"Recall: {metrics.box.re:.4f}")
except Exception:
    # If metrics object differs, print repr for inspection
    print("Metrics object:", repr(metrics))

# =========================================================
# EXPORT ZIP FILE (Optional)
# =========================================================
shutil.make_archive(save_dir, 'zip', save_dir)
print("✅ Training + Augmentation + Export completed successfully!")


✅ Random seed set to 42
Albumentations: 1.3.1
Torch: 2.6.0+cu124
Dataset directory not writable — copying to a writable location (this may take time).
Copied dataset to /kaggle/working/violenceweapondetection
Ultralytics 8.3.229 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=None, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/violenceweapondetection/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0025, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2,